In [1]:
import matplotlib.pyplot as plt
from segments.utils import get_semantic_bitmap
from segments import SegmentsClient
from segments.huggingface import release2dataset
from datasets import load_dataset

import requests
from transformers import pipeline
import numpy as np
from PIL import Image, ImageDraw

c:\Users\dskon\anaconda3\envs\compvision\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


In [9]:

#your segments.ai api key
api_key = "your_api_key"
from huggingface_hub import notebook_login

client = SegmentsClient(api_key) #initializing segments.ai client
notebook_login() #logging into HF 

NameError: name 'SegmentsClient' is not defined

In [2]:
dataset_identifier = "dskong07/chargers-full"
name = "chargers-labeled-full-v0.1"
release_name = "chargers-labeled-full-v0.1"
hub_model_id = "segformer-b0-finetuned-segments-chargers-2-15"
hf_dataset_identifier = f"dskong07/chargers-full-v0.1"
id2label = {0: 'unlabeled', 1: 'screen', 2: 'body', 3: 'cable', 4: 'plug', 5: 'void-background'}
label2id = {v: k for k, v in id2label.items()}

In [3]:
from datasets import load_dataset

ds = load_dataset(hf_dataset_identifier)

In [4]:
#creating train test

ds = ds.shuffle(seed=1)
ds = ds["train"].train_test_split(test_size=0.2)
train_ds = ds["train"]
test_ds = ds["test"]

In [5]:
from transformers import SegformerForSemanticSegmentation


pretrained_model_name = "nvidia/mit-b1" 
model = SegformerForSemanticSegmentation.from_pretrained(
    pretrained_model_name,
    id2label=id2label,
    label2id=label2id
)

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b1 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from torchvision.transforms import ColorJitter
from transformers import (
    SegformerImageProcessor,
)

processor = SegformerImageProcessor()
jitter = ColorJitter(brightness=0.25, contrast=0.25, saturation=0.25, hue=0.1) 

def train_transforms(example_batch):
    images = [jitter(x) for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = processor(images, labels)
    return inputs


def val_transforms(example_batch):
    images = [x for x in example_batch['pixel_values']]
    labels = [x for x in example_batch['label']]
    inputs = processor(images, labels)
    return inputs


# Set transforms
train_ds.set_transform(train_transforms)
test_ds.set_transform(val_transforms)

In [ ]:
from transformers import TrainingArguments

epochs = 50
lr = 0.00006
batch_size = 2

hub_model_id = "segformer-b1-finetuned-segments-chargers-full-v0.1"

training_args = TrainingArguments(
    "segformer-b1-finetuned-segments-chargers-outputs-v0.1",
    learning_rate=lr,
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    save_total_limit=3,
    evaluation_strategy="steps",
    save_strategy="steps",
    save_steps=20,
    eval_steps=20,
    logging_steps=1,
    eval_accumulation_steps=5,
    load_best_model_at_end=True,
    push_to_hub=True,
    hub_model_id=hub_model_id,
    hub_strategy="end",
)

c:\Users\dskon\anaconda3\envs\compvision\Lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [8]:
import torch
from torch import nn
import evaluate
import multiprocessing

metric = evaluate.load("mean_iou")

def compute_metrics(eval_pred):
  with torch.no_grad():
    logits, labels = eval_pred
    logits_tensor = torch.from_numpy(logits)
    # scale the logits to the size of the label
    logits_tensor = nn.functional.interpolate(
        logits_tensor,
        size=labels.shape[-2:],
        mode="bilinear",
        align_corners=False,
    ).argmax(dim=1)

    pred_labels = logits_tensor.detach().cpu().numpy()
    metrics = metric._compute(
            predictions=pred_labels,
            references=labels,
            num_labels=len(id2label),
            ignore_index=0,
            reduce_labels=processor.do_reduce_labels,
        )
    
    # add per category metrics as individual key-value pairs
    per_category_accuracy = metrics.pop("per_category_accuracy").tolist()
    per_category_iou = metrics.pop("per_category_iou").tolist()

    metrics.update({f"accuracy_{id2label[i]}": v for i, v in enumerate(per_category_accuracy)})
    metrics.update({f"iou_{id2label[i]}": v for i, v in enumerate(per_category_iou)})

    return metrics

In [9]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)

In [10]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Currently logged in as: dskong07 (dskong07-uc-san-diego) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss,Mean Iou,Mean Accuracy,Overall Accuracy,Accuracy Unlabeled,Accuracy Screen,Accuracy Body,Accuracy Cable,Accuracy Plug,Accuracy Void-background,Iou Unlabeled,Iou Screen,Iou Body,Iou Cable,Iou Plug,Iou Void-background
20,1.272200,1.648703,0.286624,0.551591,0.645251,nan,0.043285,0.908939,0.434277,0.785996,0.585458,0.000000,0.042110,0.483536,0.218094,0.409937,0.566065
40,0.988400,1.208456,0.334004,0.559426,0.736679,nan,0.041104,0.857678,0.266168,0.863150,0.769029,0.000000,0.040294,0.527733,0.188823,0.527183,0.719991
60,1.110300,1.006850,0.362892,0.585732,0.768480,nan,0.063689,0.911193,0.286406,0.871405,0.795965,0.000000,0.063657,0.574426,0.206226,0.574460,0.758585
80,0.710000,0.857612,0.525697,0.681017,0.797908,nan,0.499596,0.855381,0.365389,0.861882,0.822839,nan,0.420065,0.610181,0.249083,0.569957,0.779200
100,0.909400,0.799461,0.543541,0.684274,0.816562,nan,0.380357,0.870860,0.453026,0.861699,0.855430,nan,0.359463,0.630780,0.307238,0.613549,0.806673
120,1.077300,0.750417,0.505087,0.618026,0.807469,nan,0.077583,0.937033,0.422586,0.801233,0.851696,nan,0.077570,0.598862,0.336452,0.700513,0.812037
140,0.731300,0.632891,0.588366,0.705195,0.836905,nan,0.434232,0.814125,0.551698,0.815427,0.910492,nan,0.403887,0.628778,0.391016,0.686012,0.832136
160,0.639800,0.604064,0.604350,0.735292,0.841855,nan,0.507677,0.893563,0.588389,0.812892,0.873939,nan,0.449913,0.662883,0.413112,0.663960,0.831882
180,0.472800,0.643517,0.581553,0.715461,0.827978,nan,0.453826,0.941584,0.511125,0.834848,0.835923,nan,0.415058,0.649513,0.392014,0.639673,0.811505
200,1.102800,0.638923,0.589206,0.702277,0.835019,nan,0.421579,0.869470,0.541886,0.790747,0.887704,nan,0.398817,0.631740,0.406201,0.680787,0.828486


C:\Users\dskon\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0\mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
C:\Users\dskon\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0\mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
C:\Users\dskon\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0\mean_iou.py:260: RuntimeWarning: invalid value encountered in divide
  acc = total_area_intersect / total_area_label
C:\Users\dskon\.cache\huggingface\modules\evaluate_modules\metrics\evaluate-metric--mean_iou\9e450724f21f05592bfb0255fe2fa576df8171fa060d11121d8aecfff0db80d0\mean_iou.py:259: 

TrainOutput(global_step=670, training_loss=0.5829685669114341, metrics={'train_runtime': 1377.3634, 'train_samples_per_second': 0.966, 'train_steps_per_second': 0.486, 'total_flos': 8.58443566546944e+16, 'train_loss': 0.5829685669114341, 'epoch': 10.0})